In [9]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [7]:
data = datasets.load_breast_cancer()
X, y = data.data, data.target

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.25, shuffle=True)

In [11]:
X_train.shape, X_test.shape

((426, 30), (143, 30))

In [12]:
import numpy as np
from math import e

"""
entropy is a concept used in decision trees and related algorithms to measure the impurity or disorder of a set of instances. 
The entropy of a set is highest when all instances in the set belong to different classes,
and lowest when all instances belong to the same class. 
The entropy value is used as a metric to determine the best feature to split a set into smaller subsets, 
with the goal of reducing impurity and improving the accuracy of predictions.
"""

def _entropy(y):
    _, counts = np.unique(y, return_counts=True)
    probas = counts/counts.sum()
    size  = -e if len(counts)==1 else np.log(len(counts))
    return -np.sum(probas * np.log(probas))/size


In [18]:
"""
Information gain is a measure used in decision tree algorithms to quantify the reduction in 
entropy or impurity achieved by partitioning a set of instances based on a specific feature.
It is the difference between the entropy of the original set and the weighted average of the entropies
of the resulting subsets. The goal is to select the feature with the highest information gain,
as this would result in the largest reduction in impurity and the best split. 
Information gain is used as a criterion for selecting the best feature to split a set 
at each step of the decision tree building process.

"""

def _entropy(y):
    _, counts = np.unique(y, return_counts=True)
    probas = counts/counts.sum()
    size  = -e if len(counts)==1 else np.log(len(counts))
    return -np.sum(probas * np.log(probas))/size


def _split(X, feature, threshold):
    left_idxs = np.argwhere(X[:, feature]<=threshold).flatten()
    right_idxs = np.argwhere(X[:, feature]>threshold).flatten()
    return left_idxs, right_idxs


def _information_gain(X, y, feature, threshold):
    parent = _entropy(y)
    left_idxs, right_idxs = _split(X, feature, threshold)

    if len(left_idxs)==0 or len(right_idxs)==0:
        return 0

    l_child = y[left_idxs]
    r_child = y[right_idxs]

    n_samples = y.shape[0]
    l_samples = l_child.shape[0]
    r_samples = r_child.shape[0]

    l_entropy = _entropy(l_child)
    r_entropy = _entropy(r_child)

    ig = parent - ((l_samples/n_samples) * l_entropy) + ((l_samples/n_samples) * r_entropy)

    return ig


In [19]:
"""
The "best split" in a decision tree refers to the process of dividing a node 
(representing a set of instances) into two or more child nodes, 
based on the values of a specific feature. 
The goal of this process is to create homogeneous child nodes, 
with instances belonging to the same class as much as possible. 
The best split is found by selecting the feature that results in the largest reduction in entropy or impurity,
as measured by metrics such as information gain or Gini index. By repeatedly splitting nodes in this way,
the decision tree algorithm builds a tree-like structure that can be used for classification or regression tasks.
"""

def _determine_best_split(X, y, features):
    best_gain=-np.inf
    best_feature=None
    best_threshold=None
    for _, f in enumerate(features):
        thresholds = np.unique(X[:,f])
        for threshold in thresholds:
            ig = _information_gain(X, y, f, threshold)

            if ig>best_gain:
                best_gain=ig
                best_feature=f
                best_threshold=threshold
    return best_feature, best_threshold

In [53]:
from collections import Counter

"""
decision tree is a type of algorithm used in machine learning and artificial intelligence 
for solving problems related to classification and regression. It works by constructing a tree-like model,
where each internal node represents a test on a feature, each branch represents an outcome of the test,
and each leaf node represents a prediction (in classification) or a value (in regression). 
"""

class Node:
    def __init__(self, value=None, left=None, right=None, feature=None, threshold=None):
        self.value=value
        self.left=left
        self.right=right
        self.feature=feature
        self.threshold=threshold


def build_tree(X, y, counter=1, max_depth=3, min_samples_leaf=50, min_features=None):
    min_samples, n_features = X.shape

    features = n_features if min_features is None else min_features

    labels= np.unique(y)

    if counter>=max_depth or len(labels)==1 or min_samples<min_samples_leaf:
        leaf_val = Counter(y).most_common(1)[0][0]
        return Node(value=leaf_val)

    features = np.random.choice(n_features, features, replace=False)

    best_feature, best_value = _determine_best_split(X, y, features)

    left_idxs, right_idxs = _split(X, best_feature, best_value)

    left_child = build_tree(X[left_idxs, :], y[left_idxs], counter+1)

    right_child = build_tree(X[right_idxs, :], y[right_idxs], counter+1)

    return Node(left=left_child, right=right_child, feature=best_feature, threshold=best_value)



def predict(X, tree):
    predicted = np.array([_traverse(x, tree) for x in X])
    return predicted


def _traverse(X, tree):
    if tree.value is not None:
        return tree.value

    if X[tree.feature]<=tree.threshold:
        return _traverse(X, tree.left)

    return _traverse(X, tree.right)



In [68]:
%%time

tree = build_tree(X_train, y_train)

predictions = predict(X_test, tree)

np.sum(predictions==y_test)/y_test.shape[0]

Wall time: 5.84 s


0.7902097902097902

In [58]:
"""
Bootstrap sampling is a statistical technique used in machine learning
and other fields to estimate the uncertainty of a model's predictions.
 
It involves creating multiple samples of the training data by randomly
selecting instances with replacement, so that each sample has the same size
as the original data but may contain duplicate instances. By training a model on each bootstrapped sample,
we obtain a collection of models, each fit to a different random subset of the data.

This collection can then be used to obtain various statistical measures,
such as the mean and standard deviation of model performance,
 or to estimate the uncertainty of the model's predictions for new instances. 

Bootstrap sampling is a widely used technique for estimating the stability 
and generalization performance of machine learning models.
"""

def _bootstrap_samples(X, y, min_samples=300):
    n_samples=X.shape[0]

    min_samples=n_samples if min_samples is None else min_samples

    idxs = np.random.choice(n_samples, min_samples, replace=True)

    return X[idxs, :], y[idxs]

In [66]:
%%time

from scipy.stats import mode

"""
Random Forest is an ensemble learning method for classification and regression that operates
by constructing a collection of decision trees, where each tree is trained on a randomly selected subset 
of the training data. The predictions made by the individual trees are then combined, 
either by voting in classification or by averaging in regression, to produce a single prediction.
"""

def RandomForestClassifier(X, y, n_estimators=100):
    n_trees = []
    for _ in range(n_estimators):
        X, y = _bootstrap_samples(X, y)
        m = build_tree(X, y)
        n_trees.append(m)
    return n_trees


model = RandomForestClassifier(X_train, y_train, n_estimators=100)


def predict(X, tree):
    predicted = np.array([_traverse(x, tree) for x in X])
    return predicted



def _traverse(X, tree):
    if tree.value is not None:
        return tree.value

    if X[tree.feature]<=tree.threshold:
        return _traverse(X, tree.left)

    return _traverse(X, tree.right)


def _majority_voting(X):
    return mode(X, axis=1)[0]


def rf_predict(X):
    preds = [predict(X_test, m) for m in model]
    predicted = np.stack(preds, axis=1)
    predicted = _majority_voting(predicted)
    return predicted



Wall time: 26.5 s


In [67]:
predictions = rf_predict(X_test)

np.sum(predictions==y_test)/y_test.shape[0]

82.75524475524476